In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, inChannels, outChannels):
        super(BasicBlock, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv3d(inChannels, outChannels, kernel_size=3, padding=1),
            nn.BatchNorm3d(outChannels),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv3d(outChannels, outChannels, kernel_size=3, padding=1),
            nn.BatchNorm3d(outChannels),
            nn.ReLU(inplace=True)
        )
        self.layer3 = nn.Sequential(
            nn.Conv3d(outChannels, outChannels, kernel_size=3, padding=1),
            nn.BatchNorm3d(outChannels),
        )
        self.resLayer = nn.Sequential(
            nn.Conv3d(inChannels, outChannels, kernel_size=1),
            nn.BatchNorm3d(outChannels),
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        resOut = self.resLayer(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x + resOut
        return x

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self):
        super(AttentionBlock, self).__init__()
        self.downsample = nn.MaxPool3d(kernel_size=2, stride=2)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        o = x
        print(x.shape)
        x = self.downsample(x)
        print(x.shape)
        x = self.downsample(x)
        print(x.shape)
        x = F.interpolate(x, scale_factor=2, mode='trilinear')
        print(x.shape)
        x = F.interpolate(x, size=o.shape[2:], mode='trilinear')
        print(x.shape)
        x = self.sigmoid(x)
        return x

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.block1 = BasicBlock(1, 8)
        self.block2 = BasicBlock(8, 16)
        self.block3 = BasicBlock(16, 32)
        self.block4 = BasicBlock(32, 48)
        self.maxpool = nn.MaxPool3d(kernel_size=2, stride=2, padding=1)
        self.dropLayer = nn.Dropout(p=0.5)
        self.fc = nn.Sequential(
            nn.Linear(48 * 7 * 8 * 7, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 7)
        )
        self.relu = nn.ReLU(inplace=True)
        self.attention = AttentionBlock()
    def forward(self, x):
        x = self.block1(x)
        x = self.maxpool(x)
        x = self.block2(x)
        x_a = self.attention(x)
        x = self.relu(x.mul(x_a) + x)
        x = self.maxpool(x)
        x = self.block3(x)
        x_a = self.attention(x)
        x = self.relu(x.mul(x_a) + x)
        x = self.maxpool(x)
        x = self.block4(x)
        x_a = self.attention(x)
        x = self.relu(x.mul(x_a) + x)
        x = self.maxpool(x)
        x = self.dropLayer(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x